# Sales Dashboard

## 1. import libraries 

In [1]:
# import libraries  
import pandas as pd
import numpy as np
import datetime

## 2. Collect Data 

In [3]:
# connect to google drive 
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# import csv file from google drive
fc = pd.read_csv('/content/gdrive/MyDrive/work/new data/ahmedfisal06-04-2022@CashBook.csv')
ahcash=pd.read_csv('/content/gdrive/MyDrive/work/new data/ahmedcash01-04-2022@CashBook.csv')
ahkh=pd.read_csv('/content/gdrive/MyDrive/work/new data/ahmedbok06-04-2022@CashBook.csv')
kcash=pd.read_csv('/content/gdrive/MyDrive/work/new data/ khalidcash06-04-2022@CashBook.csv')
kkh=pd.read_csv('/content/gdrive/MyDrive/work/new data/ khalid bok06-04-2022@CashBook.csv')
omdur=pd.read_csv('/content/gdrive/MyDrive/work/new data/ omdur06-04-2022@CashBook.csv')
ahnile=pd.read_csv('/content/gdrive/MyDrive/work/new data/ ahmed nile06-04-2022@CashBook.csv')
tad=pd.read_csv('/content/gdrive/MyDrive/work/new data/tadamon 01-04-2022@CashBook.csv')
cheeck=pd.read_csv('/content/gdrive/MyDrive/work/new data/ cheek01-04-2022@CashBook.csv')
usdollar=pd.read_csv('/content/gdrive/MyDrive/DataScience/Us dollar time series forcasting/cleaned us dollar.csv')

In [ ]:
# merge data
df=pd.concat([ahkh,kcash,omdur,ahcash,ahnile,tad,cheeck,kkh,fc],axis=0)
df=df.reset_index(drop=True)
df['Date']=pd.to_datetime(df['Date'])
df.drop(["Time","Mode"],axis=1,inplace=True)
df.fillna(0,inplace=True)
df['TransID']=list(df.index)

In [ ]:
# split every remark row
# split by line
for i in range(20):
  df['remark']=df.Remark.str.split("\n|\+| و ")
  i=i+1
df=df.explode('remark').drop('Remark',axis=1).reset_index(drop=True)
df

## 3. Data cleaning and features Extraction

In [ ]:
# replace wrong values
df['remark'].replace(regex=['ماسوره'], value='ماسورة',inplace=True)
df['remark'].replace(regex=['بوصة|بةصه|بَصه|يوصه '], value='بوصه',inplace=True)
df['remark'].replace(regex=['معكوسه|معكوس |محوله|محولة'], value='معكوسة',inplace=True)
df['remark'].replace(regex=['ذكاه|زكاه|زكاة'], value='ذكاة',inplace=True)
df['remark'].replace(regex=['المعموره'], value='المعمورة',inplace=True)
df['remark'].replace(regex=['×'], value='*',inplace=True)
df['remark'].replace(regex=['١','٢','٣','٤','٥','٦','٧','٨','٩','٠'], value=['1','2','3','4','5','6','7','8','9','0'],inplace=True)
df['remark'].replace(regex=['1/2 2|21/2|2 و1/2|2 ونص|ونص 2|2ونص|2/1 2'], value='2.5',inplace=True)
df['remark'].replace(regex=['11/2|1/2 1|1ونص |11/2'], value='1.5',inplace=True)
df['remark'].replace(regex=['1/4 1|11/4|11/4'], value='1.25',inplace=True)
df['remark'].replace(regex=[' نص |١/٢'], value='0.5',inplace=True)
df['remark'].replace(regex=['٣/٤|٠.٧٥'], value='0.75',inplace=True)
df['remark'].replace(regex=['ملم|3 ملم|3 ملي|3ملي'], value='3 mm',inplace=True)

In [ ]:

# categorization
# سحب خالد تحت الحساب
khalidwd=df[df['remark'].str.contains("خالد تحت الحساب")]
# سحب احمد تحت الحساب
ahmedwd=df[df['remark'].str.contains("احمد تحت الحساب|سحب احمد")]
# منصرفات  الوالد
aboy=df[df['remark'].str.contains("ابوي|البيت|البنات| امي |مدرس |استاذ")]
# القروش المعكوسة متضمنة تحويل الزبائن 
revall=df[df['remark'].str.contains("معكوسة|مديونيه|مديونية")]
# قروش الزبائن المعكوسة مع شراء بضائع
revsell=revall[revall['remark'].str.contains("ماسورة")]
# القروش المعكوسة غير للزبائن
rev=revall[~revall['remark'].isin(revsell['remark'])]
# الذكاة
zaka=df[df['remark'].str.contains("ذكاة")]
# ايجار الوقف
wa8f=df[df['remark'].str.contains("ايجار الوقف")]
# ايجار المعمورة
mamo=df[df['remark'].str.contains("ايجار المعمورة")]
# merge all above
all=pd.concat([rev,aboy,ahmedwd,khalidwd,zaka,wa8f,mamo]).reset_index()
# البضائع
goods=df[~df['remark'].isin(all['remark'])]
# منصرفات  الوالد
aboy=aboy[~aboy.remark.str.contains('معكوسة')]
aboy=aboy[~aboy.remark.str.contains('سحب احمد تحت الحساب')]
aboy = aboy[~aboy.isin(zaka)]
aboy=aboy[aboy['Cash In']==0]
# export to excel 
aboy.to_excel('/content/gdrive/MyDrive/work/aboy.xlsx')
# export to excel 
khalidwd.to_excel('/content/gdrive/MyDrive/work/khalidexpenses.xlsx')
khalidswd=df[df['remark'].str.contains("خالد")]
khalidswd=khalidswd[df['remark'].str.contains("مصاريف")]
khalidswd.to_excel('/content/gdrive/MyDrive/work/khalidexpensesweek.xlsx')

In [ ]:
# size extraction
size=[]
for i in range(len(df.index)):
  try:
    size.append(df.remark[i].split('ماسورة')[1].split('بوصه')[0])
  except IndexError:
    size.append('null')
df['size']=size

In [ ]:
# Quantities extraction
Qt=[]
for i in range(len(df.index)):
  try:
    Qt.append(df.remark[i].split('ماسورة')[0])
  except IndexError:
    Qt.append('null')
    
df['Qt']=Qt
for i in range(len(df.index)):
  if df['remark'][i]==df['Qt'][i]:
    df.loc[i,'Qt']='null'
# fill missing values
df.loc[:,'Qt']=df.loc[:,'Qt'].replace('',1)

In [ ]:
df['price']=df[df['remark'].str.contains("ماسورة")].remark.str.extract('([*]\d+|[*] \d+)',expand=True)
df['price']=df['price'].str.extract('(\d+)',expand=True)

df['Qt']=df['Qt'].replace(regex=[' نص |١/٢'], value='0.5')

df[df['remark'].str.contains("ماسورة")]

df['price']=df['price'].fillna(0)
df['Qt']=df['Qt'].replace(regex=['نص'], value='0.5')
df['Qt']=df['Qt'].str.extract('(\d+)',expand=True)
df['Qt']=df['Qt'].fillna(1)

In [ ]:
df['Total']=np.nan
for i in df[df['remark'].str.contains("ماسورة")][df.price==0].index:
  df.loc[i,'Total']=df.loc[i,'Cash In']

In [ ]:
# replace manually
df.loc[38,'size'],df.loc[38,'Qt']=2,20
df['Qt']=df['Qt'].replace('و0',0)

In [ ]:
# type extraction 
df['type']=df[df['remark'].str.contains("ماسورة")].remark.str.extract(r'(تقيل|عادي|وسط|تايلندي|3 mm)')

for i in df[df['remark'].str.contains("ماسورة")].index:
            if df[df['remark'].str.contains("ماسورة")]['price'][i]==np.nan:
              df[df['remark'].str.contains("ماسورة")]['price'][i]=str(df[df['remark'].str.contains("ماسورة")]['Total'][i]) / (df[df['remark'].str.contains("ماسورة")]['Qt'][i])

In [ ]:
# size extraction
df.loc[df['remark'].str.contains("ماسورة"),'size']=df.loc[df['remark'].str.contains("ماسورة"),'size'].apply(lambda x: x.split('([*])')[0])
df['size']=df.loc[df['remark'].str.contains("ماسورة"),'size'].str.extract('(\d+.\d+|\d+)',expand=True)

In [ ]:
# replace manually
df.loc[df['size']=='3 3']=3
df.loc[df['size']=='3*6']=3

In [ ]:
df=df.fillna(0)
df=df.replace(['3/4','1/2'],[0.75,0.5])
df['Qt']=df['Qt'].replace('null',0)
df['price']=df['price'].astype('float')
df['Total']=df['Total'].astype('float')
df['Qt']=df['Qt'].astype('float')
df['size']=df['size'].astype('float')

# correct type
df['type']=df['type'].replace(0,'عادي')

In [ ]:
for i in df['price'][df['price']==0].index:
  try:
    df.loc[df['price']==0,'price'][i]=df['Total'].loc[df['price'][df['price']==0].index][i] / df['Qt'].loc[df['price'][df['price']==0].index][i]
  except ZeroDivisionError:
	  df.loc[df['price']==0,'price'][i] = 0
for i in df['Total'][df['Total'].isnull()].index:
  df.loc[i,'Total']=df.loc[i,'qt'] * df.loc[i,'price']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df.loc[df['remark'].str.contains("ماسورة",na=False)]

,Date,Entry by,Cash In,Cash Out,Balance,TransID,remark,size,Qt,price,Total,type
9,2021-06-27 00:00:00,Ahmedkaka,87500.0,0.0,924100,9,5ماسورة 2بوصه عادي *17500,2.0,5.0,17500.0,0.0,عادي
12,2021-06-28 00:00:00,Ahmedkaka,152000.0,0.0,1091600,12,10ماسورة 1.5قلفنايز بالجلبه,1.5,10.0,0.0,152000.0,عادي
16,2021-06-28 00:00:00,Ahmedkaka,70000.0,0.0,1207400,16,ماسورة 4بوصه مقطعه طول علي 3,4.0,1.0,0.0,70000.0,عادي
36,2021-07-05 00:00:00,Ahmedkaka,680000.0,0.0,905050,36,20ماسورة 1.5 بوصه عادي و20 2 بوصه عادي,1.5,20.0,0.0,680000.0,عادي
43,2021-07-07 00:00:00,Ahmedkaka,40000.0,0.0,1018150,43,ماسورة 2.5 بوصه,2.5,1.0,0.0,40000.0,عادي
...,...,...,...,...,...,...,...,...,...,...,...,...
4255,2021-12-02 00:00:00,Khalid,220000.0,0.0,257265,3558,10 ماسورة 2 بوصه عادي * 22000,2.0,10.0,22000.0,0.0,عادي
4266,2022-02-09 00:00:00,Ahmedkaka,152000.0,0.0,179865,3569,40 ماسورة 1/2 بوصه عادي*3800,0.5,40.0,3800.0,0.0,عادي
4270,2022-02-16 00:00:00,Khalid,190000.0,0.0,362753,3573,5 ماسورة 2.5 بوصه عادي * 38000,2.5,5.0,38000.0,0.0,عادي
4274,2022-02-20 00:00:00,Ahmedkaka,24000.0,0.0,24753,3576,نص ماسورة 3 بوصه,3.0,0.0,0.0,24000.0,عادي


In [ ]:
trans=df.loc[df['remark'].str.contains("ماسورة",na=False)]
trans.loc[:,'Qt'].replace(0.0,0.5,inplace=True)
for i in trans.loc[trans.price==0,'price'].index:
  trans.loc[i,'price']=trans.loc[i,'Total'] / trans.loc[i,'Qt']
for i in trans.loc[trans.Total==0,'Total'].index:
  trans.loc[i,'Total']=trans.loc[i,'price'] * trans.loc[i,'Qt']
trans

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Date,Entry by,Cash In,Cash Out,Balance,TransID,remark,size,Qt,price,Total,type
9,2021-06-27 00:00:00,Ahmedkaka,87500.0,0.0,924100,9,5ماسورة 2بوصه عادي *17500,2.0,5.0,17500.0,87500.0,عادي
12,2021-06-28 00:00:00,Ahmedkaka,152000.0,0.0,1091600,12,10ماسورة 1.5قلفنايز بالجلبه,1.5,10.0,15200.0,152000.0,عادي
16,2021-06-28 00:00:00,Ahmedkaka,70000.0,0.0,1207400,16,ماسورة 4بوصه مقطعه طول علي 3,4.0,1.0,70000.0,70000.0,عادي
36,2021-07-05 00:00:00,Ahmedkaka,680000.0,0.0,905050,36,20ماسورة 1.5 بوصه عادي و20 2 بوصه عادي,1.5,20.0,34000.0,680000.0,عادي
43,2021-07-07 00:00:00,Ahmedkaka,40000.0,0.0,1018150,43,ماسورة 2.5 بوصه,2.5,1.0,40000.0,40000.0,عادي
...,...,...,...,...,...,...,...,...,...,...,...,...
4255,2021-12-02 00:00:00,Khalid,220000.0,0.0,257265,3558,10 ماسورة 2 بوصه عادي * 22000,2.0,10.0,22000.0,220000.0,عادي
4266,2022-02-09 00:00:00,Ahmedkaka,152000.0,0.0,179865,3569,40 ماسورة 1/2 بوصه عادي*3800,0.5,40.0,3800.0,152000.0,عادي
4270,2022-02-16 00:00:00,Khalid,190000.0,0.0,362753,3573,5 ماسورة 2.5 بوصه عادي * 38000,2.5,5.0,38000.0,190000.0,عادي
4274,2022-02-20 00:00:00,Ahmedkaka,24000.0,0.0,24753,3576,نص ماسورة 3 بوصه,3.0,0.5,48000.0,24000.0,عادي


In [ ]:
# Products valuation
# اسعار بضاعة عمر الصادق مايو 
keys=[0.5,0.75,1,1.25,1.5,2.5,3,4]
values=[2600,3250,4100,7500,8500,16500,28500,36000]
omeraslsadig_may=pd.DataFrame(zip(keys,values))
omeraslsadig_may['type']='عادي'
omeraslsadig_may.columns=['size','cost','type']

# اسعار بضاعة مصطفى عليش  يوليو  
keys=[0.5,0.75,1,1.25,2,2.5,3,4]
values=[3250,5000,6500,5000,16500,35000,40000,50000]
eleshjul=pd.DataFrame(zip(keys,values))
eleshjul['type']='عادي'
eleshjul.columns=['size','cost','type']

# اسعار بضاعة اسماعيل النعيم
ismaelaug=pd.DataFrame({'size':[3],'cost':[34000],'type':['3 mm']})

# اسعار بضاعة عزو التيمات سبتمبر 
keys=[3,6]
values=[42000,85000]
ezo_sep=pd.DataFrame(zip(keys,values))
ezo_sep['type']='عادي'
ezo_sep.columns=['size','cost','type']

# اسعار بضاعة عمر الصادق ديسمبر 
keys=[0.5,0.75,1,1.25,1.5,2,2.5,3,4]
values=[3500.00,4500.00,5800.00,11000.00,13500.00,17500.0,30000.00,40000.00,50000.00]
values=[j-(j*0.0462) for i,j in enumerate(values)]
omeraslsadig_dec=pd.DataFrame(zip(keys,values))
omeraslsadig_dec['type']='عادي'
omeraslsadig_dec.columns=['size','cost','type']

# اسعار بضاعة عمر الحاج ديسمبر 
keys=[0.5,0.75,1,1.25,1.5,2,3,4]
values=[4200,5300,7800,11600,13600,17200,45000,55000]
values=[j-j*0.07 for i,j in enumerate(values)]
omeralhag_dec=pd.DataFrame(zip(keys,values))
omeralhag_dec['type']='عادي'
omeralhag_dec.columns=['size','cost','type']

# اسعار بضاعة عمر الصادق مارس 2022
keys=[0.5,0.75,1,1.25,1.5,2,3,4]
values=[4500,6000,8000,13000,15000,20000,50000,60000]
omeralsadigmarch=pd.DataFrame(zip(keys,values))
omeralsadigmarch['type']='عادي'
omeralsadigmarch.columns=['size','cost','type']

# اسعار بضاعة بدية السنة  العادية 
keys=[0.5,0.75,1,1.25,1.5,2,2.5,3,4,5,6]
values=[1000,2000,3000,4000,5500,7000,15000,15000,18000,25000,35000]
startnoraml=pd.DataFrame(zip(keys,values))
startnoraml['type']='عادي'
startnoraml.columns=['size','cost','type']

# اسعار بضاعة بداية السنة  التقيلة 
keys=[0.5,0.75,1,1.25,1.5,2,3,4]
values=[1800,3000,4000,5000,7000,12000,18000,20000]
starthev=pd.DataFrame(zip(keys,values))
starthev['type']='تقيل'
starthev.columns=['size','cost','type']

In [ ]:
# start year cost
x=trans[trans['type']!='عادي'].merge(starthev,how='inner',on=['size']).drop('type_y',axis=1)
x=x.rename(columns={'type_x':'type'})
y=trans.merge(startnoraml,how='inner',on=['type','size'])
trans=pd.concat([x,y],axis=0)
trans=trans.reset_index(drop=True)
trans['Date']= pd.to_datetime(trans['Date'])
trans=trans.sort_values(by='Date').reset_index(drop=True)
trans

,Date,Entry by,Cash In,Cash Out,Balance,TransID,remark,size,Qt,price,Total,type,cost
0,2021-06-26,Khalid,55000.0,0.0,1809640,2703,ماسورة 2.5 * 43000,2.5,1.0,43000.0,43000.0,عادي,15000
1,2021-06-27,Khalid,28000.0,0.0,1831140,2705,2 ماسورة 1 بوصه 3 mm *14000,1.0,2.0,14000.0,28000.0,3 mm,4000
2,2021-06-27,Ahmedkaka,87500.0,0.0,924100,9,5ماسورة 2بوصه عادي *17500,2.0,5.0,17500.0,87500.0,عادي,7000
3,2021-06-27,Ahmedkaka,57000.0,0.0,1937500,2164,ماسورة 2بوصه قلفنايز عادي مقطعه,2.0,1.0,57000.0,57000.0,عادي,7000
4,2021-06-28,Ahmedkaka,152000.0,0.0,1091600,12,10ماسورة 1.5قلفنايز بالجلبه,1.5,10.0,15200.0,152000.0,عادي,5500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,2022-04-03,Ahmedkaka,198000.0,0.0,642390,1956,1 ماسورة 3 بوصه عادي * 60000,3.0,1.0,60000.0,60000.0,عادي,15000
962,2022-04-03,Ahmedkaka,198000.0,0.0,642390,1956,2 ماسورة 4 بوصه عادي * 69000,4.0,2.0,69000.0,138000.0,عادي,18000
963,2022-04-04,Khalid,500000.0,0.0,929237,3508,100 ماسورة 1/2 بوصه عادي * 5000,0.5,100.0,5000.0,500000.0,عادي,1000
964,2022-04-05,Khalid,5500.0,0.0,575890,1965,1 ماسورة 1/2 بوصه عادي,0.5,1.0,5500.0,5500.0,عادي,1000


In [ ]:
# product cost variation through the year

# cost in may 2021
t=trans.loc[(trans['type']=='عادي')&(trans['Date'] > '2021-05-27')].merge(omeraslsadig_may,how='outer',on=['size','type'])
t['cost'] = np.max(t[['cost_x', 'cost_y']], axis=1)
t.drop(['cost_x','cost_y'],axis=1,inplace=True)

# cost in July 2021
t1=trans.loc[(trans['type']=='عادي')&(trans['Date'] > '2021-07-07')&('2021-12-05' < trans['Date'])].merge(eleshjul,how='outer',on=['size','type'])
t1['cost'] = np.max(t1[['cost_x', 'cost_y']], axis=1)
t1.drop(['cost_x','cost_y'],axis=1,inplace=True)

# cost in Aug 2021 
t2=trans.loc[(trans['type']=='3 mm')&(trans['Date'] > '2021-08-07')].merge(ismaelaug,how='outer',on=['size','type'])
t2['cost'] = np.max(t2[['cost_x', 'cost_y']], axis=1)
t2.drop(['cost_x','cost_y'],axis=1,inplace=True)

# cost in Sep 2021 
t3=trans.loc[(trans['type']=='عادي')&(trans['Date'] > '2021-09-27')].merge(ezo_sep,how='outer',on=['size','type'])
t3['cost'] = np.max(t3[['cost_x', 'cost_y']], axis=1)
t3.drop(['cost_x','cost_y'],axis=1,inplace=True)

# cost in December 2021
l=list(omeraslsadig_dec['cost'])
y=list(omeralhag_dec['cost'])
del l[-3]
c=[(g + h) / 2 for g, h in zip(l, y)]
dec=pd.DataFrame({'cost':[3622, 4610, 6393, 10639, 12762, 16343,28614, 40001, 49420],'size':list(omeraslsadig_dec['size']),'type':'عادي'})
dec['cost']=dec['cost'].astype('int')

t4=trans.loc[(trans['type']=='عادي')&(trans['Date'] > '2021-12-05')].merge(dec,how='outer',on=['size','type'])
t4['cost'] = np.max(t4[['cost_x', 'cost_y']], axis=1)
t4.drop(['cost_x','cost_y'],axis=1,inplace=True)

# cost in march 2022
t5=trans.loc[(trans['type']=='عادي')&(trans['Date'] > '2022-03-29')].merge(omeralsadigmarch,how='outer',on=['size','type'])
t5['cost'] = np.max(t5[['cost_x', 'cost_y']], axis=1)
t5.drop(['cost_x','cost_y'],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:1208: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


In [ ]:
# combine data 
trans1=pd.concat([trans,t,t1,t2,t3,t4],axis=0).sort_values(by='cost')
trans1=trans1.drop_duplicates(list(trans1.drop('cost',axis=1).columns),keep='last')
trans1=trans1.sort_values(by='Date').reset_index(drop=True)
trans1

,Date,Entry by,Cash In,Cash Out,Balance,TransID,remark,size,Qt,price,Total,type,cost
0,2021-06-26,Khalid,55000.0,0.0,1809640,2703,ماسورة 2.5 * 43000,2.5,1.0,43000.0,43000.0,عادي,16500.0
1,2021-06-27,Ahmedkaka,57000.0,0.0,1937500,2164,ماسورة 2بوصه قلفنايز عادي مقطعه,2.0,1.0,57000.0,57000.0,عادي,7000.0
2,2021-06-27,Ahmedkaka,87500.0,0.0,924100,9,5ماسورة 2بوصه عادي *17500,2.0,5.0,17500.0,87500.0,عادي,7000.0
3,2021-06-27,Khalid,28000.0,0.0,1831140,2705,2 ماسورة 1 بوصه 3 mm *14000,1.0,2.0,14000.0,28000.0,3 mm,4000.0
4,2021-06-28,Ahmedkaka,152000.0,0.0,1091600,12,10ماسورة 1.5قلفنايز بالجلبه,1.5,10.0,15200.0,152000.0,عادي,8500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,2022-04-03,Ahmedkaka,198000.0,0.0,642390,1956,1 ماسورة 3 بوصه عادي * 60000,3.0,1.0,60000.0,60000.0,عادي,42000.0
962,2022-04-03,Ahmedkaka,198000.0,0.0,642390,1956,2 ماسورة 4 بوصه عادي * 69000,4.0,2.0,69000.0,138000.0,عادي,50000.0
963,2022-04-04,Khalid,500000.0,0.0,929237,3508,100 ماسورة 1/2 بوصه عادي * 5000,0.5,100.0,5000.0,500000.0,عادي,3622.0
964,2022-04-05,Khalid,5500.0,0.0,575890,1965,1 ماسورة 1/2 بوصه عادي,0.5,1.0,5500.0,5500.0,عادي,3622.0


In [ ]:
trans=trans1
trans.Qt[625],trans.price[625],trans.Total[625]=49,65800,3224200
trans=trans.drop([442,321,307,352,919,304,811])
trans=trans[~trans['remark'].str.contains('(باقي|متبقي|راجع)')]
trans=trans[trans['Cash Out']==0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


In [ ]:
trans['Totalcost']=trans['Qt'] * trans['cost']
trans['profit']=trans['price'] - trans['cost']
trans['Total profit']=trans['Total'] - trans['Totalcost']

In [ ]:
usdollar=usdollar.rename(columns={'Datetime':'Date'})
usdollar=usdollar[['Date','dollar']].reset_index(drop=True)
usdollar['Date']=pd.to_datetime(usdollar['Date'])
trans=trans.merge(usdollar,how='left',on='Date')
trans['cost dollar']=trans['cost'] /trans['dollar']
trans['Total cost dollar']=trans['Totalcost'] /trans['dollar']
trans['profit dollar']=trans['profit'] /trans['dollar']
trans['Total profit dollar']=trans['Total profit'] /trans['dollar']

In [ ]:
trans

,Date,Entry by,Cash In,Cash Out,Balance,TransID,remark,size,Qt,price,...,type,cost,Totalcost,profit,Total profit,dollar,cost dollar,Total cost dollar,profit dollar,Total profit dollar
0,2021-06-26,Khalid,55000.0,0.0,1809640,2703,ماسورة 2.5 * 43000,2.5,1.0,43000.0,...,عادي,16500.0,16500.0,26500.0,26500.0,465.0,35.483871,35.483871,56.989247,56.989247
1,2021-06-27,Ahmedkaka,57000.0,0.0,1937500,2164,ماسورة 2بوصه قلفنايز عادي مقطعه,2.0,1.0,57000.0,...,عادي,7000.0,7000.0,50000.0,50000.0,465.0,15.053763,15.053763,107.526882,107.526882
2,2021-06-27,Ahmedkaka,87500.0,0.0,924100,9,5ماسورة 2بوصه عادي *17500,2.0,5.0,17500.0,...,عادي,7000.0,35000.0,10500.0,52500.0,465.0,15.053763,75.268817,22.580645,112.903226
3,2021-06-27,Khalid,28000.0,0.0,1831140,2705,2 ماسورة 1 بوصه 3 mm *14000,1.0,2.0,14000.0,...,3 mm,4000.0,8000.0,10000.0,20000.0,465.0,8.602151,17.204301,21.505376,43.010753
4,2021-06-28,Ahmedkaka,152000.0,0.0,1091600,12,10ماسورة 1.5قلفنايز بالجلبه,1.5,10.0,15200.0,...,عادي,8500.0,85000.0,6700.0,67000.0,465.0,18.279570,182.795699,14.408602,144.086022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,2022-04-03,Ahmedkaka,198000.0,0.0,642390,1956,1 ماسورة 3 بوصه عادي * 60000,3.0,1.0,60000.0,...,عادي,42000.0,42000.0,18000.0,18000.0,575.0,73.043478,73.043478,31.304348,31.304348
812,2022-04-03,Ahmedkaka,198000.0,0.0,642390,1956,2 ماسورة 4 بوصه عادي * 69000,4.0,2.0,69000.0,...,عادي,50000.0,100000.0,19000.0,38000.0,575.0,86.956522,173.913043,33.043478,66.086957
813,2022-04-04,Khalid,500000.0,0.0,929237,3508,100 ماسورة 1/2 بوصه عادي * 5000,0.5,100.0,5000.0,...,عادي,3622.0,362200.0,1378.0,137800.0,575.0,6.299130,629.913043,2.396522,239.652174
814,2022-04-05,Khalid,5500.0,0.0,575890,1965,1 ماسورة 1/2 بوصه عادي,0.5,1.0,5500.0,...,عادي,3622.0,3622.0,1878.0,1878.0,575.0,6.299130,6.299130,3.266087,3.266087


In [ ]:
# save to excel file
trans.to_excel('/content/gdrive/MyDrive/work/transaction.xlsx')

# Create model excel model file
df=trans[['Date','size','Total','type']]
df.to_excel('/content/gdrive/MyDrive/work/model.xlsx')